## Import required libraries

In [15]:
import requests
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import folium

# Part 1: Getting and cleaning the data

## Scraping Wikipedia page to get Toronto Neighbourhoods dataframe

In [16]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [17]:
r = requests.get(url) #do I need custom header here?

In [18]:
dfs=pd.read_html(r.text, header=0)

In [19]:
toronto_neis = dfs[0]
toronto_neis.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [20]:
toronto_neis.shape

(180, 3)

## Let's clean the dataframe

In [21]:
toronto_neis['Borough'].value_counts()

Not assigned        77
North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
York                 5
East Toronto         5
East York            5
Mississauga          1
Name: Borough, dtype: int64

In [22]:
toronto_neis = toronto_neis[toronto_neis.Borough != 'Not assigned']
toronto_neis.shape

(103, 3)

In [23]:
toronto_neis.loc[toronto_neis['Neighbourhood'] == 'Not Assigned']

,Postal Code,Borough,Neighbourhood


In [24]:
toronto_neis.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


# Part 2: Geocoding

## Let's have coordinates in or neighbourhoods dataframe

In [25]:
geo_csv = pd.read_csv('Geospatial_Coordinates.csv')

In [26]:
geo_csv.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [27]:
toronto_neis_geocoded = pd.merge(toronto_neis, geo_csv, on='Postal Code')
toronto_neis_geocoded.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## Let's see the map

In [28]:
#Toronto coordinates
address = 'Toronto'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [31]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(toronto_neis_geocoded['Latitude'], toronto_neis_geocoded['Longitude'], toronto_neis_geocoded['Borough'], toronto_neis_geocoded['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# Part 3: Clustering